In [18]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.nasnet import NASNetMobile
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### Prepare Dataset

In [37]:
TRAIN_DIR = "../../data/SantaCruzIsland_Labeled_5Class"
BATCH_SIZE = 8
NUM_EPOCHS = 10
WIDTH = 1920
HEIGHT = 1080
num_train_images = 100

# The total size of training dataset
totakl_train_size = num_train_images * NUM_EPOCHS

In [4]:
train_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=90, \
                                         horizontal_flip=True, vertical_flip=True)

In [5]:
train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(WIDTH, HEIGHT), batch_size = BATCH_SIZE)

Found 1554 images belonging to 5 classes.


### Create Model

In [6]:
resnet50_model = ResNet50(input_shape=(WIDTH, HEIGHT, 3), weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


/home/michael/miniconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [7]:
nasnet_mobile = NASNetMobile(include_top=False, weights='imagenet', pooling='max')

In [8]:
def build_model(base_model, dropout, fc_layers, num_classes):

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # let's add fully-connected layers
    # Every FC layer has a dropout probability
    for fc in fc_layers:
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)

    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)    
    
    return model

In [9]:
class_list = ["Birds", "Empty", "Fox", "Humans", "Rodents"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

layered_model = build_model(resnet50_model, dropout=dropout, fc_layers=FC_LAYERS, num_classes=len(class_list))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Train the model

In [35]:
def train_model(model, model_name, data_generator):

    # create adam optimizer with learning rate
    adam = Adam(lr=0.00001)

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    
    checkpoint = ModelCheckpoint("../checkpoints/{0}_model_weights.h5".format(model_name), monitor=["acc"], verbose=1, mode='max')
    
    history = model.fit_generator(data_generator, epochs = NUM_EPOCHS, workers=8, 
                                  steps_per_epoch=num_train_images // BATCH_SIZE, 
                                  shuffle=True, callbacks=[checkpoint])

    return history
    
    

In [11]:
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    
    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    plt.figure()
    plt.plot(epochs, loss, 'r.')
    plt.plot(epochs, val_loss, 'r')
    plt.title('Training and validation loss')
    plt.show()
    
    plt.savefig('training_history.png')
    

In [38]:
history = train_model(layered_model, "ResNet50", train_generator)

Epoch 1/10
12/12 [==============================] - 34s 3s/step - loss: 0.4672 - acc: 0.8125

Epoch 00001: saving model to ../checkpoints/ResNet50_model_weights.h5
Epoch 2/10
12/12 [==============================] - 14s 1s/step - loss: 0.6488 - acc: 0.7188

Epoch 00002: saving model to ../checkpoints/ResNet50_model_weights.h5
Epoch 3/10
12/12 [==============================] - 17s 1s/step - loss: 0.6025 - acc: 0.7396

Epoch 00003: saving model to ../checkpoints/ResNet50_model_weights.h5
Epoch 4/10
12/12 [==============================] - 20s 2s/step - loss: 0.7294 - acc: 0.7188

Epoch 00004: saving model to ../checkpoints/ResNet50_model_weights.h5
Epoch 5/10
12/12 [==============================] - 17s 1s/step - loss: 0.6787 - acc: 0.7083

Epoch 00005: saving model to ../checkpoints/ResNet50_model_weights.h5
Epoch 6/10
12/12 [==============================] - 21s 2s/step - loss: 0.6555 - acc: 0.7188

Epoch 00006: saving model to ../checkpoints/ResNet50_model_weights.h5
Epoch 7/10
12/12

In [39]:
history.history


{'loss': [0.46720914045969647,
  0.6487811456123987,
  0.6025203764438629,
  0.7293557077646255,
  0.6786509106556574,
  0.6554749086499214,
  0.5657738695542017,
  0.729920523862044,
  0.6290730039278666,
  0.5553552731871605],
 'acc': [0.8125,
  0.71875,
  0.7395833333333334,
  0.71875,
  0.7083333333333334,
  0.71875,
  0.75,
  0.7083333333333334,
  0.6770833333333334,
  0.7291666666666666]}